In [1]:
import pandas as pd
import torch
import sklearn
import numpy as np

from transformers import AutoTokenizer, AutoConfig, EarlyStoppingCallback, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

df = pd.read_csv("../dataset/train/train.csv")
df

,id,sentence,subject_entity,object_entity,label,source
0,0,〈Something〉는 조지 해리슨이 쓰고 비틀즈가 1969년 앨범 《Abbey R...,"{'word': '비틀즈', 'start_idx': 24, 'end_idx': 26...","{'word': '조지 해리슨', 'start_idx': 13, 'end_idx':...",no_relation,wikipedia
1,1,호남이 기반인 바른미래당·대안신당·민주평화당이 우여곡절 끝에 합당해 민생당(가칭)으...,"{'word': '민주평화당', 'start_idx': 19, 'end_idx': ...","{'word': '대안신당', 'start_idx': 14, 'end_idx': 1...",no_relation,wikitree
2,2,K리그2에서 성적 1위를 달리고 있는 광주FC는 지난 26일 한국프로축구연맹으로부터...,"{'word': '광주FC', 'start_idx': 21, 'end_idx': 2...","{'word': '한국프로축구연맹', 'start_idx': 34, 'end_idx...",org:member_of,wikitree
3,3,균일가 생활용품점 (주)아성다이소(대표 박정부)는 코로나19 바이러스로 어려움을 겪...,"{'word': '아성다이소', 'start_idx': 13, 'end_idx': ...","{'word': '박정부', 'start_idx': 22, 'end_idx': 24...",org:top_members/employees,wikitree
4,4,1967년 프로 야구 드래프트 1순위로 요미우리 자이언츠에게 입단하면서 등번호는 8...,"{'word': '요미우리 자이언츠', 'start_idx': 22, 'end_id...","{'word': '1967', 'start_idx': 0, 'end_idx': 3,...",no_relation,wikipedia
...,...,...,...,...,...,...
32465,32465,한국당은 7일 오전 9시부터 오후 5시까지 진행된 원내대표 및 정책위의장 후보자 등...,"{'word': '유기준', 'start_idx': 93, 'end_idx': 95...","{'word': '부산 서구·동구', 'start_idx': 100, 'end_id...",per:employee_of,wikitree
32466,32466,"법포는 다시 최시형, 서병학, 손병희 직계인 북접과 다시 서장옥, 전봉준, 김개남을...","{'word': '최시형', 'start_idx': 7, 'end_idx': 9, ...","{'word': '손병희', 'start_idx': 17, 'end_idx': 19...",per:colleagues,wikipedia
32467,32467,완도군(군수 신우철)이 국토교통부에서 실시한 '2019 교통문화지수 실태조사'에서 ...,"{'word': '완도군', 'start_idx': 0, 'end_idx': 2, ...","{'word': '신우철', 'start_idx': 7, 'end_idx': 9, ...",org:top_members/employees,wikitree
32468,32468,"중앙일보, JTBC 회장을 지낸 이후 중앙홀딩스 회장, 재단법인 한반도평화만들기 이...","{'word': 'JTBC', 'start_idx': 6, 'end_idx': 9,...","{'word': '중앙홀딩스', 'start_idx': 21, 'end_idx': ...",no_relation,wikipedia


In [2]:
print(df.iloc[32284]['sentence'])
print(df.iloc[32284]['subject_entity'])
print(df.iloc[32284]['object_entity'])
print(df.iloc[32284]['label'])

초반에는 도쿄에서 한 마을인 히나미자와 마을에 전학 온지 얼마 안된 소년 마에바라 케이이치와 개성적인 동아리의 동료들과의 일상을 그리지만 중반 이후에 갑자기 발생하는 괴사건과 서스펜스가 공포 분위기를 두드러지게 한다.
{'word': '마에바라 케이이치', 'start_idx': 41, 'end_idx': 49, 'type': 'PER'}
{'word': '히나미자와', 'start_idx': 16, 'end_idx': 20, 'type': 'LOC'}
per:colleagues


In [4]:
for i in df.index:
    if df['label'].iloc[i] == "per:colleagues":
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() == "'ORG'" or df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() == "'ORG'":
            print(df.iloc[i]['sentence'], i)

서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다. 2321
29일 TBS '김어준의 뉴스공장'에 유시민 노무현재단 이사장이 출연했다. 3358
유시민 노무현재단 이사장이 백년이 지나도 한국의 언론개혁이 이루어 지지 않을 것으로 내다봤다. 4430
강인은 "무엇보다 14년이란 오랜 시간 동안 언제나 과분한 사랑을 주신 E.L.F. 여러분들께 가장 죄송한 마음입니다"라며 "많이 늦었지만 슈퍼주니어란 이름을 내려놓고 홀로 걷는 길에도 항상 미안함과 고마움을 가슴에 새기고 나아가겠습니다"라고 했다. 4534
8월 29일, 유시민 노무현재단 이사장은 서울대학교 촛불 집회에 대하여 "뒤에서 자유한국당 손길이 어른어른하는 거라고 본다"며, 왜 마스크를 쓰고 집회하냐고 발언하여 논란을 빚었다. 7754
그다음 시즌, 카이저슬라우테른은 1947 시즌에 프리츠 발터와 오트마어 발터에 큰힘을 입어 북부 조 (Gruppe Nord) 에서 손쉽게 우승을 거두었다. 13812
유시민 노무현재단 이사장이 조국 사태에 "인간이 무섭다는 생각이 든다"고 말했다. 17550
와타나베는 원래 [[연병관]](練兵館)의 우두머리로, 조슈 번의 중신 [[기도 다카요시]]나 [[다카스기 신사쿠]] 등과 막역한 사이였기 때문에, 료마는 와타나베에게 조슈 번에 로비를 의뢰했다. 20409
유시민 노무현재단 이사장이 KBS 법조팀과 검찰의 유착 의혹을 제기했다. 22518
산투스 복귀 후, 호비뉴는 발뒤꿈치로 그의 영입을 갈망했던 상파울루를 상대로 경기 종료 5분을 남기고 득점했고, "물고기 군단"은 2-1 승리를 쟁취했다. 25367


In [5]:
print(df.iloc[2321]['sentence'])
print(df.iloc[2321]['subject_entity'])
print(df.iloc[2321]['object_entity'])
print(df.iloc[2321]['label'])

서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다.
{'word': '유시민', 'start_idx': 12, 'end_idx': 14, 'type': 'PER'}
{'word': '노무현', 'start_idx': 16, 'end_idx': 18, 'type': 'ORG'}
per:colleagues


### 사람 - 사람 관계이어야 하는 것들
이때에 POH를 추가해주어 기타 명칭이 될 수 있는 것들도 포함을 해주었다.

In [6]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:colleagues" or df['label'].iloc[i] == "per:siblings" or df['label'].iloc[i] == "per:children" or df['label'].iloc[i] == "per:parents" or df['label'].iloc[i] == "per:spouse" or df['label'].iloc[i] == "per:other_family":
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'"):
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

1725년(영조 1) 2월 18일에 화순옹주로 봉해졌으며, 1732년(영조 8) 영의정을 지낸 김흥경(金興慶)의 아들이자 추사 김정희의 증조부인 월성위(月城尉) 김한신(金漢藎)과 혼례를 치렀다. 708
양평군의 어머니 숙의 이씨는 양성 이씨 출신으로, 중종 때 판의금부사와 좌찬성 등을 역임한 이손과 일가 친척이다. 843
아버지는 덴마크 왕 덴마크의 크리스티안 9세, 어머니는 헤센카셀의 루이세이다. 1504
그의 아들 성삼문, 성삼빙, 성삼고, 성삼성과 손자 네 명이 모두 처형당하여 그의 후사는 끊겼다. 가까운 친척으로는 그의 사촌 성희의 후손들이 살아남아 현존하고 있다. 1729
로이히텐베르크의 요세피나는 조제핀이라는 이름으로 외젠 드 보아르네와 바이에른 공주 아우구스테의 장녀로 태어났다. 1992
서병수 전 부산시장이 유시민 노무현재단 이사장을 향해 쓴소리했다. 2321
29일 TBS '김어준의 뉴스공장'에 유시민 노무현재단 이사장이 출연했다. 3358
공빈 김씨(恭嬪 金氏, 1553년 11월 16일 ~ 1577년 6월 13일) 또는 공성왕후(恭聖王后)는 조선의 제 14대 임금 선조의 빈이자 추존왕후이며 임해군, 광해군 형제의 생모이다. 3382
마우리아-셀레우코스 전쟁 이후 찬드라굽타 마우리아는 인도 아대륙 안쪽으로 관심을 돌려 북인도뿐만 아니라 중부 인도와 남인도 지역에도 확장을 시도하였고 그 결과 마우리아 제국은 오디샤 지역의 칼링가와 타밀라캄 지역의 촐라, 판디아, 체라를 제외한 인도 아대륙 대부분을 통일하였지만, 찬드라굽타 마우리아 재위 말기인 기원전 298년에 마우리아 제국의 영토에 발생한 극심한 가뭄으로 인해 수많은 사람들이 기아로 사망하는 일이 발생하자 이에 충격을 받은 찬드라굽타 마우리아는 당시 그가 스승으로 모시던 자이나교 구루의 충고를 들은 후 정치에 손을 떼고 아들인 빈두사라에게 마우리아 제국의 황제 자리를 물려주었으며, 자이나교에 귀의하여 출가한 후 고행을 하다가 일생을 마쳤다. 4077
남연군 이구와 여흥군부인 여흥 민씨의 셋째 아

In [7]:
print(df.iloc[32320]['sentence'])
print(df.iloc[32320]['subject_entity'])
print(df.iloc[32320]['object_entity'])
print(df.iloc[32320]['label'])

칭기즈 칸의 씨족인 보르지긴은 황금씨족(알탄 우룩)이라 불리며, 영지 백성(우르스)으로 나뉜 천호・백호・십호 집단의 위에 상급 영주 계급으로써 군림했고, 몽골 황제 즉 대칸은 크고 작은 우르스의 가장 큰 부분을 가진 맹주였다.
{'word': '칭기즈 칸', 'start_idx': 0, 'end_idx': 4, 'type': 'PER'}
{'word': '보르지긴', 'start_idx': 11, 'end_idx': 14, 'type': 'ORG'}
per:other_family


정말로 이상한 것을 확인할 수 있었음

### 사람 - Date 관계가 이상한 것들

In [8]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:date_of_birth" or df['label'].iloc[i] == "per:date_of_death":
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'DAT'"):
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

발레리나 출신인 최혜영씨는 1979년생으로 무용학과를 졸업했지만 2003년 교통사고로 사지마비 척수장애를 입고 무용수의 꿈을 접고, 여성 척수장애인 국내 최초로 재활학 박사가 됐다. 3033
동치제(同治帝, 함풍(咸豊)(咸豊) 6년 3월 23일(1856년 4월 27일) ~ 동치(同治) 13년 12월 5일(1875년 1월 12일))은 중국 청 왕조의 제10대 황제(재위 1861년 ~ 1874년)이다. 6671
광서제(光緖帝, 동치(同治)(同治) 10년 6월 28일(1871년 8월 14일) ~ 광서(光緖) 34년 10월 21일(1908년 11월 14일))는 청나라의 제11대 황제이다. 14741
홍희제(洪熙帝, 홍무(洪武)(洪武) 11년 7월 23일(1378년 8월 16일) ~ 홍희(洪熙) 원년 5월 12일(1425년 5월 29일))는 중국 명나라의 제4대 황제(재위 1424년~1425년)이다. 18279
페르시아 제국의 다리우스 3세가 알렉산드로스 대왕에게 패하자 그를 살해한 박트리아의 총독 베수스는 민족적 저항을 조직하려고 하였다. 19531
우키타 히데이에(겐키 3년(1572년)(1572년) ~ 메이레키 원년(1655년 12월 17일))는 아즈치모모야마 시대의 비젠 오카야마성 다이묘이다. 20742
브라마굽타(597–668)는 펠 방정식의 연구를 시작하였으며, 자야데바(Jayadeva, 9세기 경)가 펠 방정식의 해를 완성하였다. 22631
쩐 영종(바오푸(寶符)(寶符) 4년 9월 17일(1276년 10월 27일) ~ 다이카인(大慶) 7년 3월 16일(1320년 4월 24일))은 베트남 쩐 왕조의 제4대 황제(재위 : 1293년 ~ 1314년)이다. 27484
하지만, 김춘추와 문희의 첫번째 아들인 김법민(문무왕)이 626년생으로, 문희가 김법민을 임신했을때는 선덕여왕이 즉위하기 5년전인 진평왕 때라는 것이 된다. 27954
요기 베라는 자신의 메이저 데뷔 일과 같은 날로 내보였던 2015년 9월 22일에는 뉴저지주 웨스트 칼드웰에 있는 고령자 집합 주택에서

In [10]:
print(df.iloc[3033]['sentence'])
print(df.iloc[3033]['subject_entity'])
print(df.iloc[3033]['object_entity'])
print(df.iloc[3033]['label'])

발레리나 출신인 최혜영씨는 1979년생으로 무용학과를 졸업했지만 2003년 교통사고로 사지마비 척수장애를 입고 무용수의 꿈을 접고, 여성 척수장애인 국내 최초로 재활학 박사가 됐다.
{'word': '최혜영', 'start_idx': 9, 'end_idx': 11, 'type': 'PER'}
{'word': '1979년', 'start_idx': 15, 'end_idx': 19, 'type': 'NOH'}
per:date_of_birth


In [11]:
print(df.iloc[6671]['sentence'])
print(df.iloc[6671]['subject_entity'])
print(df.iloc[6671]['object_entity'])
print(df.iloc[6671]['label'])

동치제(同治帝, 함풍(咸豊)(咸豊) 6년 3월 23일(1856년 4월 27일) ~ 동치(同治) 13년 12월 5일(1875년 1월 12일))은 중국 청 왕조의 제10대 황제(재위 1861년 ~ 1874년)이다.
{'word': '동치제', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}
{'word': '동치(同治) 13년 12월 5일(1875년 1월 12일)', 'start_idx': 46, 'end_idx': 76, 'type': 'PER'}
per:date_of_death


## 사람 데이터의 경우
### PER Subject Entity가 이상한 것

In [20]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:title" or df['label'].iloc[i] == "per:employee_of" or df['label'].iloc[i] == "per:product" or df['label'].iloc[i] == "per:chidren" or df['label'].iloc[i] == "per:place_of_residence" or df['label'].iloc[i] == "per:alternate_names" or df['label'].iloc[i] == "per:other_family" or df['label'].iloc[i] == "per:colleagues" or df['label'].iloc[i] == "per:origin" or df['label'].iloc[i] == "per:siblings" or df['label'].iloc[i] == "per:spouse" or df['label'].iloc[i] == "per:parents" or df['label'].iloc[i] == "per:schoools_attended" or df['label'].iloc[i] == "per:date_of_death" or df['label'].iloc[i] == "per:birth" or df['label'].iloc[i] == "per:palce_of_birth" or df['label'].iloc[i] == "per:place_of_death" or df['label'].iloc[i] == "per:religion":
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'":
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

0


위의 관계는 없는 것으로 보인다.

### 사람과 LOC  관계이어야 하는 것들

In [34]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:place_of_residence" or df['label'].iloc[i] == "per:place_of_death" or df['label'].iloc[i] == "per:place_of_birth" or df['label'].iloc[i] == "per:schools_attended" :
        if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'":
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

우중문은 을지문덕이나 고구려왕이 올 때에는 반드시 잡으라는 양제의 기밀명령에 따라 을지문덕을 잡으려 했으나, 우문술, 유사룡 등 주변 장수들이 항복한 장수를 잡아들이면 저들은 목숨을 걸고 싸울 것이라며 반대하였다. 2520
666년 12월, 고구려가 형제간 내부 권력투쟁이 발생하는 동안 연개소문의 동생이자,연남생,연남건 형제의 숙부인 고구려의 대신 연정토가 고구려 남쪽의 12성, 7백 63호, 3543명을 데리고 신라에 투항 해 버렸다. 5182
선의왕후 어씨는 본관이 함종(咸從)으로 영조 6년(1730) 6월 29일 경희궁 어조당(魚藻堂)에서 소생없이 승하하니 춘추 26세였고, 시호를 선의(宣懿), 무덤의 이름을 의릉이라 하여 10월에 천장산 경종릉 아래에 장례를 지냈다. 11019
원숭환(袁崇煥, 1584년 - 1630년)은 명나라 말기의 무장이다. 12857
이스마일 2세(;, 쿰 ~, 카즈빈)은 세 번째 사파비 제국의 군주이다. 13184
인천 계양구는 4일, 박형우 계양구청장을 비롯하여 박남춘 인천광역시장, 계양구의회 의원, 지역구 국회의원, 인천시 의원, 유관단체장 및 입주 업체들과 함께 서운일반산업단지 준공식을 개최했다. 15825
그러나 이스마일 2세는 , 카즈빈의 궁성에서 아편 중독으로 사망하였다. 15970
스테판 무고샤는 FK 부두치노스트 포드고리차에서 축구를 시작하여, 2009년 몬테네그로 1부 리그에서 데뷔 무대를 가졌다. 19638
‘2020 대한민국 수공예공모대전’에서 강진군 청자업체인 토우 김유성 대표가 `청자 연리문 금채 다기`를 출품해 금상을 수상했다. 20813
먼저 제갈량은 오장원으로 가서 그곳을 점거하고 위수 남쪽에는 맹염에게 진을 치도록 했다. 22530
백제의 멸망에 대한 내용이 가사에 담겨 있는 이 노래는 조선총독부가 발매금지 조치를 취했고, 후에 작사가 조명암이 조선민주주의인민공화국으로 가면서 대한민국에서도 금지곡이 되었다. 26270
이날 협약식에는 경남대 박재규 총장과 백두현 고성군수를 비롯해 경남대 강재관 산학

In [45]:
print(df.iloc[27643]['sentence'])
print(df.iloc[27643]['subject_entity'])
print(df.iloc[27643]['object_entity'])
print(df.iloc[27643]['label'])

이날 협약식에는 경남대 박재규 총장과 백두현 고성군수를 비롯해 경남대 강재관 산학부총장, 홍선영 대외교류처장, 이상훈 인재개발처장, 김현준 건강항노화 센터장과 고성군 이종일 미래산업과장 및 관계자가 참석했다.
{'word': '백두현', 'start_idx': 21, 'end_idx': 23, 'type': 'PER'}
{'word': '고성군', 'start_idx': 25, 'end_idx': 27, 'type': 'POH'}
per:place_of_residence


In [35]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "per:place_of_residence" or df['label'].iloc[i] == "per:place_of_death" or df['label'].iloc[i] == "per:place_of_birth":
        if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'" :
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

전쟁은 일본이 개항하면서 에도 막부(도쿠가와 막부)의 이이 나오스케(히코네 번)가 고메이 천황의 칙허도 없이 5개의 국가(미국, 네덜란드, 러시아, 영국, 프랑스)과의 안세이 5개국 조약을 체결하였다는 불평등 조약으로 인한 반막부 세력의 불만에서 기원되었다. 315
이탈리아는 반파시스트주의자인 피에트로 바돌리오 내각이 종전 직전인 1945년 7월 14일에 대일 선전포고를 하였지만, 초대받지 못했다. 648
우중문은 을지문덕이나 고구려왕이 올 때에는 반드시 잡으라는 양제의 기밀명령에 따라 을지문덕을 잡으려 했으나, 우문술, 유사룡 등 주변 장수들이 항복한 장수를 잡아들이면 저들은 목숨을 걸고 싸울 것이라며 반대하였다. 2520
전반 5분, 세네갈의 엘 하지 디우프가 프랑스 진영 오른쪽을 쇄도하며 페널티 에어리어로 패스를 했고 그걸 살리프 디아오가 받아 슛을 날렸으나 바르테즈 골키퍼의 정면으로 갔다. 3126
김창완과 그의 두 형제들로 구성되었던 그룹 산울림의 멤버이자 3형제 중 막내였던 김창익이 2008년 1월 캐나다 벤쿠버에서 제설작업 도중 사고로 사망하게 되었고, 같은 해 김창완은 산울림의 공식활동 중단소식과 아울러 젊은 연주자 4명과 함께 팀을 꾸려 음악활동을 시작하겠다는 소식을 전했다. 4421
666년 12월, 고구려가 형제간 내부 권력투쟁이 발생하는 동안 연개소문의 동생이자,연남생,연남건 형제의 숙부인 고구려의 대신 연정토가 고구려 남쪽의 12성, 7백 63호, 3543명을 데리고 신라에 투항 해 버렸다. 5182
원나라에서 일 칸국으로 파견되어 라시드 앗 딘의 《집사》의 편찬을 도왔던 볼라드 칭상(Bolad Chingsang)이나, 일 칸국에서 원나라로 파견되어 쿠빌라이 칸 휘하에서 중용되었던 자말 앗 딘(Jamal al-Dìn)ㆍ이사 켈레메치(Isa Kelemechi) 등은 그러한 칸국 간 다양한 교류의 구체적인 사례들에 해당되는 인물들이라고 할 수 있을 것이다. 8307
디에고 로페스 로드리게스(1981년 11월 3일, 갈리시아 지방 파라델라 ~

### 사람과 POH의 관계

작성된 기준으로 보았을 때는 POH로 모두 변경해주어야 하기는 한다.    
근데 그 갯수가 너무 많아서 일단은 두개 다 설정해두었다.

In [29]:
k=0
try:
    for i in df.index:
        if df['label'].iloc[i] == "per:alternate_names" or df['label'].iloc[i] == "per:title":
            if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'":
                print(df.iloc[i]['sentence'], i)
                k+=1
except:
    pass
print(k)

KT 이사회가 전원합의로 구현모 KT 커스터머&미디어부문장(사장)을 차기 회장 후보로 확정했다. 136
4월 18일, 스기타는 전선시찰중인 야마모토 이소로쿠 연합함대 사령장관의 호송기를 호위하는 임무를 맡고 비행하던 중, 미군 P38기의 공격으로 교전하나 야마모토 사령관의 비행기가 격추되자 다른 6기의 동료와 함께 라바울로 후퇴하였다. 221
김중수 한국은행 총재는 “가계부채 증가 속도가 소득보다 빠른데다 질적 측면에서도 악화하고 있다. 빚 총량 뿐 아니라 취약한 가계부채 구조를 개선하려는 노력을 지속해야 한다”고 말했다. 226
그의 어머니인 소피아는 당시 스웨덴의 국왕인 칼 12세의 여동생으로 카를 프리드리히는 스웨덴 국왕의 외조카였다. 272
로버트 팰컨 스콧(Robert Falcon Scott, CVO, 1868년 6월 6일 ~ 1912년 3월 29일)은 영국의 군인, 탐험가이다. 444
윤상은 걸그룹 러블리즈(Lovelyz)의 아버지라고 표현될 정도로 원피스의 곡들은 대부분 러블리즈의 앨범에 집중되어 있다. 471
정경두 국방부 장관은 "코로나19가 군 내부에 확산하지 않도록 질병관리본부 등 관련 기관과 긴밀하게 공조한 가운데, 특단의 방역대책을 강구할 것"을 지시했다. 589
안소미 씨는 지난 2009년 KBS 24기 공채 개그맨으로 데뷔했다. 663
아키모토 가쓰히로(秋元 克廣) 삿포로시장은 IOC의 결정에 환영한다는 성명을 발표한 반면 고이케 유리코(小池 百合子) 도쿄도지사는 강하게 반발하여 오전 6시에서 오전 5시에서 개최할 것을 주장했다. 692
정종제 광주광역시 행정부시장이 16일 오전 시청 3층 중회의실에서 열린 1월 중 공공기관 부기관장 회의에서 기관별 주요 현안을 보고 받고 있다. 797
조석준(趙錫俊, 1954년 12월 18일 ~)은 대한민국의 기업가 겸 관료이자 예비역 대한민국 공군 대위이고 前 기자이며 前 기상행정공직자다. 828
알렉스 레나토 이바라 미나(Alex Renato Ibarra Mina, 1991년 1월 20일, 암부

In [25]:
print(df.iloc[663]['sentence'])
print(df.iloc[663]['subject_entity'])
print(df.iloc[663]['object_entity'])
print(df.iloc[663]['label'])

안소미 씨는 지난 2009년 KBS 24기 공채 개그맨으로 데뷔했다.
{'word': '안소미', 'start_idx': 0, 'end_idx': 2, 'type': 'PER'}
{'word': 'KBS 24기 공채 개그맨', 'start_idx': 16, 'end_idx': 29, 'type': 'NOH'}
per:title


In [33]:
k=0
try:
    for i in df.index:
        if df['label'].iloc[i] == "per:alternate_names" or df['label'].iloc[i] == "per:title":
            if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'":
                print(df.iloc[i]['sentence'], i)
                k+=1
except:
    pass
print(k)

2010년에는 아시아 가수 최초로 마이클 잭슨의 곡을 리메이크하였는데 당시 마이클 잭슨과 함께 작업했던 세계적인 뮤지션 스티브 바라캇(Steve Barakatt)과 마이클 잭슨 곡 "You are not alone"을 작업해 화제가 되었다. 21
에피소드에서는 외계 시간여행자인 닥터 (크리스토퍼 에클스턴)이 새로운 동행자 로즈 타일러 (빌리 파이퍼)를 타디스를 이용한 첫번째 시공간 여행으로 5,000,000년에 데려간다. 46
KT 이사회가 전원합의로 구현모 KT 커스터머&미디어부문장(사장)을 차기 회장 후보로 확정했다. 136
탤런트 강은탁(37·신슬기) 어머니 A씨가 200명에게 사기 혐의로 피소당했다. 162
2013년 방탄소년단 멤버 랩몬스터(김남준), 슈가(민윤기)는 비프리와 '김봉현의 힙합 초대석 1주년 공개방송'에 출연했다. 164
'코로나19'로 사회적 분위기가 암울한 가운데 배우 김우빈(김현중)이 한 줄기 빛 같은 소식을 전했다. 169
쇼니 씨(少弐氏)의 8대 당주로 쇼니 요리히사(少弐頼尚)의 둘째 아들이다. 191
4월 18일, 스기타는 전선시찰중인 야마모토 이소로쿠 연합함대 사령장관의 호송기를 호위하는 임무를 맡고 비행하던 중, 미군 P38기의 공격으로 교전하나 야마모토 사령관의 비행기가 격추되자 다른 6기의 동료와 함께 라바울로 후퇴하였다. 221
김중수 한국은행 총재는 “가계부채 증가 속도가 소득보다 빠른데다 질적 측면에서도 악화하고 있다. 빚 총량 뿐 아니라 취약한 가계부채 구조를 개선하려는 노력을 지속해야 한다”고 말했다. 226
조반니는 교황 레오 10세(조반니 디 로렌초 데 메디치)의 고용으로, 콘도티에로 또는 용병대장이 되어 1516년 3월 5일 프란체스코 마리아 1세 델라 로베레를 상대로 한 전쟁을 이끌었다. 268
그의 어머니인 소피아는 당시 스웨덴의 국왕인 칼 12세의 여동생으로 카를 프리드리히는 스웨덴 국왕의 외조카였다. 272
MC딩동(40·허용운)이 'MC 지망생 폭행' 의혹에 전면 부인했다. 299
신인 배우 차인

### 사람이 다녀야 하는 학교의 경우

In [36]:
k=0
try:
    for i in df.index:
        if df['label'].iloc[i] == "per:schools_attended" :
            if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'":
                print(df.iloc[i]['sentence'], i)
                k+=1
except:
    pass
print(k)

0


일단은 없는 것 같다

## ORG 부분
### ORG Subject Entity가 이상한 것

In [37]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "org:top_members_employees"  or df['label'].iloc[i] == "org:members" or df['label'].iloc[i] == "org:product" or df['label'].iloc[i] == "org:place_of_headquarters" or df['label'].iloc[i] == "org:founded" or df['label'].iloc[i] == "org:dissolved" or df['label'].iloc[i] == "org:number_of_employees" or df['label'].iloc[i] == "org:founded_by" or df['label'].iloc[i] == "org:alternate_names" or df['label'].iloc[i] == "org:political/religious_affiliation":
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'":
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

2010년 1월부터 깃허브는 GitHub, Inc. 라는 이름으로 운영되고 있다. 1465
이후 태연, 써니, 효연, 유리, 윤아로 소녀시대의 두 번째 유닛 그룹 소녀시대-Oh!GG를 결성하였으며, 9월 5일 첫 싱글 앨범 《몰랐니(Lil' Touch)》를 발표했다. 3118
1921년 하심가 출신인 압둘라 1세를 영입하여 트란스요르단을 세웠고, 제2차 세계 대전 이후인 1946년 영국으로부터 독립하였다. 21482
이렇게 싹튼 우크라이나 민족주의는 19세기 초 키예프를 중심으로 제정 러시아의 지배에서로의 해방을 목적으로 하는 정치적 운동으로 발전하였는데 여기서 주요 인물은 우크라이나의 민족 시인 타라스 셰우첸코이다. 28891
4


### ORG와 사람간의 관계

In [38]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "org:top_members/employee" :
        if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'":
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

0


이 경우는 pass

### ORG와 ORG/PER의 경우(founded_by)

In [35]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "org:founded_by" :
        if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'":
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

1921년 하심가 출신인 압둘라 1세를 영입하여 트란스요르단을 세웠고, 제2차 세계 대전 이후인 1946년 영국으로부터 독립하였다. 21482
Trans요르단는 1921년 Hashemite 가문에서 압둘라 1세을 모집하여 설립되었으며 2차 세계 대전 후 1946년 영국에서 독립했습니다. 36348
1921년, 하심가 출신의 압둘라 1세을 획득해, 트랜스요르단를 세워, 제2차 세계대전 이후의 1946년, 영국으로부터 독립했다. 42326
3


이 경우도 없다.

### ORG와 ORG간의 관계(members)

In [46]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "org:members" :
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'"and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'"):
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

이후 태연, 써니, 효연, 유리, 윤아로 소녀시대의 두 번째 유닛 그룹 소녀시대-Oh!GG를 결성하였으며, 9월 5일 첫 싱글 앨범 《몰랐니(Lil' Touch)》를 발표했다. 3118
2001년 3월 28일부터 2003년 12월 24일까지 바실리예프는 러시아 내무부 차관직을 수행하였고, 모스크바 극장 인질극 사건 때 인질들의 구출을 위한 책임을 맡았다. 18065
1990년에는 올랭피크 드 마르세유로 이적했고 1990-91 시즌에서는 올랭피크 드 마르세유의 디비지옹 1(현재의 리그 1) 우승, 유러피언컵(현재의 UEFA 챔피언스리그) 준우승에 기여했다. 19704
펠릭스클레르 리델(Félix-Clair Ridel, 1830년 7월 7일 ~ 1884년)은 프랑스 파리 외방전교회 소속의 선교사이자 천주교 조선교구의 6대교구장이며 트라키아 필리포폴리스의 명의 주교였다. 23361
당시 비틀즈의 판권을 가지고 있는 애플 레코드가 음악에 대한 모든 상표권을 가지고 있었기 때문에 애플과 분쟁이 있었고, 애플은 GS를 끝으로 미디 음원을 포기하기로 했다. 29766
비록 ICOMOS의 권고 시점에서 기준(4)의 적용은 부정되었고 세계문화유산 위원회에서조차 그 결정을 번복하지는 못했지만, 히라이즈미는 세계문화유산 등록 기준에 있어서 다음과 같은 기준을 충족했다고 간주되어 마침내 등록이 결정된다. 30202
이후 태연, 써니, 효연, 유리, 윤아는 9월 5일 소녀시대의 두 번째 유닛 그룹 소녀시대-Oh!GG를 결성하고 첫 싱글 앨범 Lil' Touch를 발매했다. 33046
2001년 3월 28일부터 2003년 12월 24일까지 Vasilyev는 러시아 내무부 차관을 맡았고 모스크바는 극장 인질 사건 중 인질 구출을 담당했습니다. 35739
1990년 그는 올랭피크 드 마르세유으로 이적했고 1990-91 시즌에는 올랭피크 드 마르세유의 디비전 1(현재 리그 1) 우승과 유러피언 컵(현재 UEFA 챔피언스 리그) 준우승에 기여했습니다. 36042
펠릭스 클레어 리델(Félix

In [58]:
k=0
for i in df.index:
    if df['label'].iloc[i] == "org:founded" or df['label'].iloc[i] == "org:dissolved" :
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'"  or df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'DAT'" :
            print(df.iloc[i]['sentence'], i)
            k+=1
print(k)

0


## 실제 코드를 구현하기 위한 부분

In [2]:
import pandas as pd
import torch
import sklearn
import numpy as np

from transformers import AutoTokenizer, AutoConfig, EarlyStoppingCallback, AutoModelForSequenceClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer

df = pd.read_csv("../dataset/train/concat_train_ratio_2.csv")

In [60]:
import re

for i in df.index:
    # 얘는 걍 이상한 애들
    if i == 3118 or i == 33046:
        df['label'].iloc[i] = "no_relation"
        df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'ORG'", df.iloc[i]['subject_entity'])
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'POH'", df.iloc[i]['object_entity'])
        
    if i == 29357:
        df['label'].iloc[i] = "per:place_of_death"
    
    if i == 21482 or i == 36348 or i == 42326:
        
        temp = df['subject_entity'].iloc[i]
        df['subject_entity'].iloc[i] = df['object_entity'].iloc[i]
        df['object_entity'].iloc[i] = temp
    
    if i == 28891:
        df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'LOC'", df.iloc[i]['subject_entity'])
        df['label'].iloc[i] = "no_relation"
    

    if i == 22569:
        df['label'].iloc[i] = "no_relation"
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'LOC'", df.iloc[i]['object_entity'])
    
    if i == 43966:
        df['label'].iloc[i] = "org:founded_by"

    if i == 19704 or i == 36042 or i == 42007:
        df['label'].iloc[i] = "org:member_of"
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'ORG'", df.iloc[i]['object_entity'])
    if i == 35739 or i == 18065:
        df['label'].iloc[i] = "org:members"
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'LOC'", df.iloc[i]['object_entity'])
    
    if i == 29766 or i == 43885:
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'ORG'", df.iloc[i]['object_entity'])

    if i == 42715 or i == 36727:
        df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'ORG'", df.iloc[i]['object_entity'])

    # 사람과 사람의 관계

    if df['label'].iloc[i] == "per:colleagues" or df['label'].iloc[i] == "per:siblings" or df['label'].iloc[i] == "per:children" or df['label'].iloc[i] == "per:parents" or df['label'].iloc[i] == "per:spouse" or df['label'].iloc[i] == "per:other_family":
        
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'"):
            
            df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'PER'", df.iloc[i]['subject_entity'])
            df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'PER'", df.iloc[i]['object_entity'])
    
    # 사람과 DAT의 관계

    if df['label'].iloc[i] == "per:date_of_birth" or df['label'].iloc[i] == "per:date_of_death":
        if (df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'" and df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'") or (df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'DAT'"):
            df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'DAT'", df.iloc[i]['object_entity'])
    
    # 사람과 LOC 관계

    if df['label'].iloc[i] == "per:place_of_residence" or df['label'].iloc[i] == "per:place_of_death" or df['label'].iloc[i] == "per:place_of_birth" or df['label'].iloc[i] == "per:schools_attended" :
        if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'LOC'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'":
            
            df['object_entity'].iloc[i] = re.sub(df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1], " 'LOC'", df.iloc[i]['object_entity'])
    
    # 사람과 POH + PER의 관계
    try:
        if df['label'].iloc[i] == "per:alternate_names" or df['label'].iloc[i] == "per:title":
            if df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'POH'" and df.iloc[i]['object_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'PER'":
                
                df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'POH'", df.iloc[i]['subject_entity'])
    except:
        pass
    
    # ORG 태깅 이상한 애들

    if df['label'].iloc[i] == "org:top_members_employees"  or df['label'].iloc[i] == "org:members" or df['label'].iloc[i] == "org:product" or df['label'].iloc[i] == "org:place_of_headquarters" or df['label'].iloc[i] == "org:founded" or df['label'].iloc[i] == "org:dissolved" or df['label'].iloc[i] == "org:number_of_employees" or df['label'].iloc[i] == "org:founded_by" or df['label'].iloc[i] == "org:alternate_names" or df['label'].iloc[i] == "org:political/religious_affiliation":
        
        if df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1].strip() != "'ORG'":
            
            df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'ORG'", df.iloc[i]['subject_entity'])
    

df.to_csv('./preprocess.csv', index=False)



/tmp/ipykernel_1392/781582594.py:71: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subject_entity'].iloc[i] = re.sub(df.iloc[i]['subject_entity'][1:-1].split(',')[3].split(':')[1], " 'POH'", df.iloc[i]['subject_entity'])
/tmp/ipykernel_1392/781582594.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'].iloc[i] = "no_relation"
/tmp/ipykernel_1392/781582594.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['subject_en

In [54]:
print(df.iloc[42715]['sentence'])
print(df.iloc[42715]['subject_entity'])
print(df.iloc[42715]['object_entity'])
print(df.iloc[42715]['label'])

펠릭스 크렐 리델(Félix-Clair Ridel, 1830년 7월 7일~1884년)은 프랑스 파리 외국전교회 소속의 선교사이며 천주교 조선교구의 6대교구장이며 트라키아 필리포폴리스 .
{'word': '천주교 조선교구', 'start_idx': 74, 'end_idx': 81, 'type': 'ORG'}
{'word': '대교구', 'start_idx': 85, 'end_idx': 87, 'type': 'NOH'}
org:members


In [6]:
import pandas as pd
import torch
import sklearn
import numpy as np

df1 = pd.read_csv("./prediction/final/submission_final_roberta_large_more.csv")
df2 = pd.read_csv("/opt/ml/git/level2-klue-level2-nlp-03/eunki/code/sota/end_point/prediction/submission_final_roberta_large_10epoch_end.csv")
k = 0
for i in df1.index:
    if df1.iloc[i]['pred_label'] != df2.iloc[i]['pred_label']:
        k += 1

In [7]:
print(k)

460


In [5]:
df1.index

RangeIndex(start=0, stop=7765, step=1)